# https://www.kaggle.com/c/titanic

In [245]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [246]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [247]:
PATH = "data/titanic/"

In [248]:
!ls {PATH}

test.csv  train.csv


In [285]:
df_raw = pd.read_csv(f'{PATH}train.csv')

In [301]:
df_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,-1,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,-1,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,-1,2


In [288]:
train_cats(df_raw)

In [289]:
df_raw.Cabin = df_raw.Cabin.cat.codes

In [290]:
df_raw.Embarked = df_raw.Embarked.cat.codes

In [291]:
df_raw.Sex = df_raw.Sex.cat.codes

In [418]:
df_raw.isnull().sum().sort_index() / len(df_raw)

Age            0.198653
Cabin          0.000000
Embarked       0.000000
Fare           0.000000
Name           0.000000
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
dtype: float64

In [303]:
df_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,-1,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,-1,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,-1,2


In [257]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/titanic-raw')

In [304]:
df, y, nas = proc_df(df_raw, 'Survived')

In [419]:
df.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Age_na         0
dtype: int64

In [421]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df, y)
m.score(df, y)

1.0

In [422]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [423]:
n_valid = 491 # kaggle test set size

n_trn = len(df) - n_valid
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

In [424]:
def print_scores(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [425]:
m = RandomForestClassifier(n_jobs=-1)
%time m.fit(X_train, y_train)
print_scores(m)

CPU times: user 154 ms, sys: 56.4 ms, total: 211 ms
Wall time: 142 ms
[1.0, 0.7983706720977597]


In [346]:
df_test_raw = pd.read_csv(f'{PATH}test.csv')

In [347]:
df_test, _, _ = proc_df(df_test_raw)

In [361]:
y_test = m.predict(df_test.drop('Fare_na', 1)) # idea: i could retrofit the model to have a 'Fare_na' column

In [401]:
d = {'PassengerId': df_test.PassengerId, 'Survived': y_test}
pd.DataFrame(d).to_csv(f'{PATH}res.csv', index=False)